In [143]:
from Crypto.Util import number


RSA is based on the fact that multiplying p by q is easy, but factoring n 
is hard. The relation between the public (e) and the private (d) exponents is 
given by $\phi(n)$ that can only be calculated if you know p and q.
The correctness of the algorithm can be proven by Fermat's little theorem.

In RSA, we select two prime numbers of equal length (p and q), and then multiply these to give a modulus:

$ N=p*q $

with

$q<p<2q$

We then compute the cipher, encrypt a message $(M)$, as:

$ C=M^e\pmod n $

and where we decrypt with:

$ M=C^d\pmod n $

We then pick e so that it does not share a factor with  $\phi$:

$ GCD(e,\phi)=1 $

and where GCD is the greatest common denominator. In practice e typically has a value of 65537 (and which is a prime number), we could select a range of values of e. If we select a fairly large value, then the value of d could be discovered.

With this we have a public exponent of e, and a private exponent of d. The value of d is computed from:

$d=e^{−1}\pmod \phi$

and where

$ \phi=(p−1)(q−1) $

In [144]:
# My secret message
msg = 'this is my secret'
msg = 'Hello, 12345 is my secret number sequence'

data = msg.encode('utf-8')
M = number.bytes_to_long(data)

print("m:", msg)

m: Hello, 12345 is my secret number sequence


## This is how the keys is generated

In [145]:
# Choose 2 large prime numbers p and q.
# For demo purposes to make print more readable
bitsize=256
p = number.getPrime(bitsize)
q = number.getPrime(bitsize)
# p = number.getStrongPrime(512)
# q = number.getStrongPrime(512)

# check they are not equal
assert(p != q)

# make sure that p is smaller than q
if p < q:
    (p, q) = (q, p)

# simple checks
assert(number.isPrime(p))
assert(number.isPrime(q))
assert(q < p)
assert(p < 2*q)

print("p:", p)
print("q:", q)
print("Prime size: %d bits" % (bitsize))


p: 83575291310042628155952311380099113663958380773616186789601585143183835336309
q: 78918097984010462692418058425047342170993220638104265006978206075658415818403
Prime size: 256 bits


In [151]:
# Setting up the encryption
# Find first part of the public key, n = p∗q.
n = p*q
phi = (p-1)*(q-1)


print("n:", n)
print("phi:", phi)
print("Encryption max size: %d bits" % (bitsize*2))
print("Encryption size: %d bits" % (n.bit_length()))


# simple check
assert(p*q == n)


n: 6595603028648162274595048349509155620306225157790753005024476281753649902178696689718374667712876448796332478984623562307161087322986602757211280976294527
phi: 6595603028648162274595048349509155620306225157790753005024476281753649902178534196329080614622028078426527332528788610705749366871190022965992438725139816
Encryption size: 511 bits
Encryption max size: 512 bits


### Key Generation

In [147]:
# generate public key 1<e<phi(n)
# The usual choice for e is F4=65537 = 0x10001. Also, having chosen e, it is simpler to test whether gcd(e,p-1)=1 and gcd(e,q-1)=1 while generating and testing the primes in step 1. Values of p or q that fail this test can be rejected there and then.
# https://www.di-mgt.com.au/rsa_alg.html#note2


e = 65537
# e = 0x10001
# e = number.getPrime(bitsize)
print("e:", e)

# "e" must be coprime with phi(n).
# You can check if e and phi(n) are coprimes using GCD(e, phi(n)) == 1.


assert(e > 1)
assert(number.GCD(e, n) == 1)
assert(number.GCD(e, p - 1) == 1)
assert(number.GCD(e, q - 1) == 1)
assert(number.GCD(e, phi) == 1)


e: 65537


In [148]:
# generate private key d
# Choose d the multiplicative inverse of e
d = pow(e, -1, phi)

print("d:", d)

# simple check
assert(d != 1)
assert(d != e)


d: 6257454712792114160664911747980999572818105217449008187640674150031229533969432850529521421111034069797613408527129551976004064634481011611097728956360809


In [149]:
# Encryption and Decryption
#

# The cipher c is the encrypted form of the message and is sent to the message receiver.
c=pow(M,e,n)

# Upon receipt of the cipher, the receiver decrypts the message with their private key
msg_plaintext = pow(c,d,n)
msg_decryptedtext = number.long_to_bytes(msg_plaintext).decode('utf-8')

print('\nPublic key (e,n) for the encryption is:\ne: %d\nn: %d' %(e,n))
print('\nPrivate key (d,n) for the decryption is:\nd: %d\nn: %d' %(d,n))
print('\nCipher: %s' % c)

print('\nSummery:')
print("Original message:", msg)
print("Plain message:", M)
print("Encrypted cipher message:", c)
print("Plain message agine:", msg_plaintext)
print("Decrypted message:", msg_decryptedtext)



print(f'Bob uses RSA to send an encrypted message to Alice. The public exponent (e) is {e} and the modulus (N) is {n}. With a cipher of {c}, determine the decrypted message:')


Public key (e,n) for the encryption is:
e: 65537
n: 6595603028648162274595048349509155620306225157790753005024476281753649902178696689718374667712876448796332478984623562307161087322986602757211280976294527

Private key (d,n) for the decryption is:
d: 6257454712792114160664911747980999572818105217449008187640674150031229533969432850529521421111034069797613408527129551976004064634481011611097728956360809
n: 6595603028648162274595048349509155620306225157790753005024476281753649902178696689718374667712876448796332478984623562307161087322986602757211280976294527

Cipher: 3005577784853804419504729110337317201013508833905234407116109845909101899763500993245336813841517623748984354013119853690237241767640261184555228557905618

Summery:
Original message: Hello, 12345 is my secret number sequence
Plain message: 154637314025019744702789137565131337379293936220748634457054852998475649718103525793122439486399333
Encrypted cipher message: 30055777848538044195047291103373172010135088339052344071161

## Code Summery

In [152]:
# Setting up the encryption
p = number.getStrongPrime(512)
q = number.getStrongPrime(512)

# key generation
n = p*q
phi = (p-1)*(q-1)

e = 65537
d = pow(e, -1, phi)
assert(number.GCD(d,n) == 1)

# Encryption
c=pow(M,e,n)
# Decryption
msg_plaintext = pow(c,d,n)
msg_decryptedtext = number.long_to_bytes(msg_plaintext).decode('utf-8')

print('Checks equality:', msg == msg_decryptedtext)

Checks equality: True


## Code examples links
- https://asecuritysite.com/rsa/rsa_full
- https://asecuritysite.com/rsa/rsa_crt4
- https://asecuritysite.com/rsa/rsa_ctf04
- https://asecuritysite.com/rsa/rsa_ctf05
- https://asecuritysite.com/rsa/rsa_12_2
- https://asecuritysite.com/cracking/rsa_ctf04
- https://mathybit.github.io/crypto-rsa/
- https://ctftime.org/writeup/29741
- https://cryptobook.nakov.com/asymmetric-key-ciphers/rsa-encrypt-decrypt-examples
- https://crypto.stackexchange.com/questions/12255/in-rsa-why-is-it-important-to-choose-e-so-that-it-is-coprime-to-%CF%86n